<a href="https://colab.research.google.com/github/weissd2k/Yunan-2021-22/blob/main/yanan-jupyterNBtestdjw.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
"""
@author: Yanan Xiong
and this is a test 
"""

import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from scipy.optimize import curve_fit,least_squares
from matplotlib.pyplot import MultipleLocator
import warnings
import datetime

#Definitions of Global variables
C0 = 1 # initial concentration of adsorbate
Cs = 40.0 # initial concentration of adsorbent
t = np.array([0,
5,
10,
15,
20,
25,
30,
35,
40])
qt = np.array([0,
0.011238,
0.014984,
0.01873,
0.022476,
0.0239744,
0.0239744,
0.0239744,
0.0239744])
dt=0.001

warnings.filterwarnings('ignore')

#### PSO takes the form:
\begin{equation}\label{eq1}
    \frac{dq_{t}}{dt} = k_{2}(q_{e} - q_{t})^{2}
\end{equation}
where $t$ is the time (minutes), $q_{t}$ is the amount of adsorbate adsorbed per mass of adsorbent at time $t$ (mg g$^{-1}$), and $q_{e}$ is the amount of adsorbate adsorbed at equilibrium (mg g$^{-1}$).

When apply the boundary conditions $t=0$ and $t=t$, $qt=0$ and $qt=qt$ and integrate eq1, we can have:
\begin{equation}\label{eq4}
    q_t = \frac{q_{e}^{2}kt}{1+q_{e}t}
\end{equation}

#### linearized PSO
The first approach towards the adsorption kinetics uses the linearized form of the integrated PSO rate equation
\begin{equation}\label{eq5}
    \frac{t}{q_{t}} = \frac{1}{k_{2}q_{e}^{2}} + \frac{t}{q_{e}}
\end{equation}
which can be regarded as a linear form of $y = mx + c$. $\frac{t}{q_{t}}$ is plotted as a function of $t$, where $m$ is the slope of the linear regression and $c$ is the y-intercept. The linear regression was implement using the built-in function $LinearRegression()$ from $sklearn$ package, which minimising the residual sum of squares between the data points and the linear regression. Hence, the equilibrium adsorption parameter $q_{e} = \frac{1}{m}$ and the $k_{2} = \frac{1}{cq_{e}^{2}}$.

#### nonlinearized PSO
Secondly, the nonlinear PSO kinetics were calculated, via an in-built function named $curve\_solve$ from the $SciPy$ package, which minimizes the sum of squared residuals between the model and experimental data, and has function has no limit to the type of curve. The initial guess applied for $curve\_solve$ is the results obtained from the linear regression.

In [4]:
### equation to get q by PSO

def PSO_getQ(t,qe,k2):
    return t/((t/qe)+(1/(k2*qe**2)))

### linear PSO

def PSO_linear(t,qt):
    # get the m and c for the linearized curve
    t_qt = t[1:]/qt[1:]
    t1=t.reshape(-1,1)
    model = LinearRegression()
    model.fit(t1[1:],t_qt)
    c = float(model.intercept_)
    m = float(model.coef_)
    
    # calculate the parameters (qe and k2) from the c,m
    qe = 1/m
    k2 = 1/(c*qe*qe)
    
    # calculate the r^2 value
    q_model = PSO_getQ(t,qe,k2)
    r_sq = 1 - ((qt-q_model)**2).sum()/((qt-qt.mean())**2).sum()

    
    return qe,k2,r_sq
### nonlinear PSO

def PSO_nonlinear(t,qt):
    # use the qe,k2 from linear method as initial guess
    p1,p2,_ = PSO_linear(t,qt)
    p0 = np.array([p1,p2])
    # find the optimised parameters qe,k2 fitting the PSO equation,
    # by using the given t,qt
    popt,pcov = curve_fit(PSO_getQ, t, qt, p0=p0)
    qe,k2 = popt
    
    # calculate the r^2 value 
    q_model = PSO_getQ(t,qe,k2)
    r_sq = 1 - ((qt-q_model)**2).sum()/((qt-qt.mean())**2).sum()
    
    return qe,k2,r_sq

### revised PSO
#### The concentration of aqueous adsorbate left at time $t$ is 
\begin{equation}\label{eq2}
    C_{t} = C_{0} - C_{s}q_{t}
\end{equation}
where $C_{t}$ is the concentration of the aqueous adsorbate at time $t$ (mg L$^{-1}$), $C_{0}$ is the initial adsorbate concentration at $t$ = 0 (mg L$^{-1}$), and {$C_{s}$} is the concentration of the adsorbent (g L$^{-1}$).

#### revised PSO takes the form: 
\begin{equation}\label{eq3}
    \frac{dq_{t}}{dt} = k'C_{t} (1 - \frac{q_{t}}{q_{e}})^{2}
\end{equation}
where the unit of rate constant $k'$ is L g$^{-1}$min$^{-1}$.
For including the operation conditions $C_{0}$ and $C_{t}$ in the equation, the $k'$ here substitutes the $\frac{k_{2}q_{e}^{2}}{C_{0}^{2}}$, combining the eq1 and eq2 together. 

Thirdly, the kinetics were obtained from the revised rate equation eq3. Due to the difficulty of integration of the rPSO, Bullen expressed the adsorbed adsorbate at the $n$th data point as 
\begin{equation}\label{eq6}
    q_{n} = q_{n-1} + ((t_{n} - t_{n-1})(k'C_{t(n-1)}(1-\frac{q{n-1}}{q_{e}})^{2}))
\end{equation}
The rPSO parameters $k'$ and $q_{e}$ were obtained by fitting the model to the experimental data via $curve_solve$ to minimize the sum of squared residuals. Both results from the linear and nonlinear PSO were used as the initial guess. They show the same results for the used set of experimental data.


In [5]:
### equation to get q by revised PSO

def rPSO_getQ(t,qe,kprime):
    # create time series with dt timestep
    t1 = np.arange(0,t[-1]+dt,dt)
    t_index = np.array(t*(1/dt),'i')
    # calculate qt for each timestep
    q = np.zeros(len(t1),dtype="float")
    q_model = []
    for i in range(1,len(t1)):
        q[i] = q[i-1]+(dt*(kprime*(C0-Cs*q[i-1])*(1-q[i-1]/qe)**2))
    # extract data points corresponding to the experimental data
    for j in t_index:
        q_model.append(q[j])
    return q_model

def rPSO_nonlinear(t,qt):           

    p1,p2,_ = PSO_nonlinear(t,qt)
    p1 = float(p1)
    p2 = float(p2*p1**2/C0**2)
    p0 = np.array([p1,p2])
    
    # find the optimised parameters qe,k2 fitting the PSO equation,
    # by using the given t,qt
    popt,pcov = curve_fit(rPSO_getQ, t, qt, p0=p0,bounds=([0.00000001,0],np.inf))
    qe,kprime = popt
    
    # calculate the r^2 value 
    q_model = rPSO_getQ(t,qe,kprime)
    r_sq = 1 - ((qt-q_model)**2).sum()/((qt-qt.mean())**2).sum()
    return qe,kprime,r_sq



In [6]:
def MonteCarlo(q_model,qe,k,func):
    # create the array to store the parameters obtained by the two MonteCarlo simulations
    k_MC = np.zeros(200)
    qe_MC = np.zeros(200)
    
    size = np.size(t)
    SSE = np.sum(np.square(q_model-qt))
    std = np.sqrt(SSE/(size-2))
    q_monte = np.zeros(size)
    
    # repeat the Monte Carlo simulation for 200 times 
    for i in range(200):
        q_monte = q_model + np.random.normal(0, std, size = (size,))
        # put the simulated data into models to obtain the parameters
        qe_MC[i],k_MC[i],_ = func(t,q_monte)
        
    # qe and k in confidence level of 95%
    qe_MC = sorted(qe_MC)
    k_MC = sorted(k_MC)
    qe_CL = qe_MC[5:195]
    k_CL = k_MC[5:195]

    # uncertainties
    qe_un = (qe_CL[-1]-qe_CL[0])/2
    k_un = (k_CL[-1]-k_CL[0])/2
    
    return qe_un,k_un

### Results

In [7]:
print("manna: linearised PSO")
qe,k2,r_sq= PSO_linear(t,qt)
q_model= t/((t/qe)+(1/(k2*qe**2)))
qe_un,k2_un = MonteCarlo(q_model,qe,k2,PSO_linear)
print(f"qe: {'%.6f'%qe} +- {'%.6f'%qe_un}")
print(f"k2: {'%.6f'%k2} +- {'%.6f'%k2_un}")
print(f"R^2: {'%.6f'%r_sq}")
print(f"qt_model: {q_model}")

manna: linearised PSO
qe: 0.029932 +- 0.003022
k2: 4.036879 +- 1.719304
R^2: 0.986769
qt_model: [0.         0.01127323 0.01637809 0.01928975 0.02117167 0.02248805
 0.0234605  0.02420825 0.0248011 ]


In [8]:
print("manna: non linearised PSO")
qe,k2,r_sq= PSO_nonlinear(t,qt)
q_model= t/((t/qe)+(1/(k2*qe**2)))
qe_un,k2_un = MonteCarlo(q_model,qe,k2,PSO_nonlinear)
print(f"qe: {'%.6f'%qe} +- {'%.6f'%qe_un}")
print(f"k2: {'%.6f'%k2} +- {'%.6f'%k2_un}")
print(f"R^2: {'%.6f'%r_sq}")
print(f"qt_model: {q_model}")

manna: non linearised PSO
qe: 0.030751 +- 0.002407
k2: 3.609864 +- 1.277394
R^2: 0.987404
qt_model: [0.         0.01097567 0.01617728 0.01921231 0.0212011  0.02260509
 0.02364916 0.024456   0.0250982 ]


In [ ]:
starttime = datetime.datetime.now()
print('manna: nonlinear rPSO')
qe,kprime,r_sq= rPSO_nonlinear(t,qt)
q_model= rPSO_getQ(t,qe,kprime)
qe_un,kprime_un = MonteCarlo(q_model,qe,kprime,rPSO_nonlinear)
print(f"qe: {'%.6f'%qe} +- {'%.6f'%qe_un}")
print(f"k2: {'%.6f'%kprime} +- {'%.6f'%kprime_un}")
print(f"R^2: {'%.6f'%r_sq}")
print(f"qt_model: {q_model}")
endtime = datetime.datetime.now()
print(endtime - starttime)

manna: nonlinear rPSO


### Visualized as plots

In [ ]:
plt.figure(figsize = (12,5),label="Manna PSO")
plt.figure(1)
ax1 = plt.subplot(131)
plt.title('linear PSO')
plt.scatter(t,qt,label = "observed data")
qe,k2,r_sq= PSO_linear(t,qt)
q_model= t/((t/qe)+(1/(k2*qe**2)))
plt.plot(t,q_model,"red",label = "linear PSO")
ax1.legend(loc='upper left')
ax1.legend("R^2: {r_sq}")
ax1.legend(fontsize=6)
ax1.set_xlabel('Time (mins)')
ax1.set_ylabel('qt (mg g-1)')
new_ticks = np.linspace(0, 1.1*qt.max(), 10)
plt.yticks(new_ticks)

ax2 = plt.subplot(132)
plt.title('non linear PSO')
plt.scatter(t,qt,label = "observed data")
qe,k2,r_sq= PSO_nonlinear(t,qt)
q_model= t/((t/qe)+(1/(k2*qe**2)))
plt.plot(t,q_model,'red', label ="non linear PSO")
ax2.legend(loc='upper left')
ax2.legend("R^2: {r_sq}")
ax2.legend(fontsize=6)
ax2.set_xlabel('Time (mins)')
ax2.set_ylabel('qt (mg g-1)')
new_ticks = np.linspace(0, 1.1*qt.max(), 10)
plt.yticks(new_ticks)

ax3 = plt.subplot(133)
plt.title('non linear rPSO')
plt.scatter(t,qt,label = "observed data")
qe,kprime,r_sq= rPSO_nonlinear(t,qt)
q_model= rPSO_getQ(t,qe,kprime)
plt.plot(t,q_model,"green",label ="non linear rPSO")
ax3.legend(loc='upper left')
ax3.legend("R^2: {r_sq}")
ax3.legend(fontsize=6)
ax3.set_xlabel('Time (mins)')
ax3.set_ylabel('qt (mg g-1)')
new_ticks = np.linspace(0, 1.1*qt.max(), 10)
plt.yticks(new_ticks)
plt.show()

### 5 Aug
#### Progress so far
1. latest code to solve the three models and Uncertainty analysis(Monte Carlo)
2. Excel to record the results for datasets
3. plots: experiment vs models
4. plots: relative errors of model results

#### Question:
1. [Cs] data 5: wrong C0 value - causing negative Ct
https://link.springer.com/article/10.1007/s11356-012-0984-z

2. There are minor difference for rPSO between excel and code, about 0.03%, which may be caused by desimal places. Jay says <1% is fine in chemistry. 

3. The relative erros of the model results show a pattern. Ususlly the at the beginning of experiment, the rPSO model shows smaller error than other models. However, the rPSO usually show a little bigger than other models, esprcially when the experiment time is long. I am wondering I should split the data into groups (ie, t<40 and t>40). Is there any proper spliting standard?
